In [1]:
import hyper_param as hp

In [5]:
price_history = hp.get_price_history()

In [7]:
print price_history.shape

(2593, 1)


In [15]:
def get_price_history():
    import pandas as pd
    df = pd.read_csv('ltc.csv')
    df[['price(USD)']] = df[['price(USD)']].fillna(method="bfill")
    return df.loc[:, ['price(USD)']].values

In [26]:
prices = get_price_history().squeeze()

In [27]:
# write a function that: take 10 price (... current price), returns the next price

In [28]:
# check: buy if next price is bigger than the current price -> buy, else, sell

In [46]:
start_t = 1000
max_t = len(prices) - 2 # -2 so that can see the next price from the last t
t = start_t

In [47]:
price = prices[t-10:t] # from t-10 to t - 1 inclusive

In [48]:
print price

[ 9.73  9.61  9.2   9.09  9.27  9.2   8.97  8.96  7.98  8.07]


In [50]:
print prices[2592]

51.48


In [51]:
BUY = -1
SELL = 1
HOLD = 0

In [59]:
# ideal policy of 1 step:
# assuming perfect knowledge of next price (if the neural net will output 100 accuracy of wheter price increase or decrease)
# if price will increase, then buy to hedge, sell in next time step.
# if price will decrease, then sell to gain, buy in next time step.
# else take action based on dynamics of price.


s = (start_t, 0, 0, None) # (t, num_coins_hold, money_so_far, next_action); 
# next_action = None: then take action based on dynamics of price, else take the specified next action
# action: -1: buy, 1: sell, 0: hold (probably not used for now)
for i in range(1000): #(max_t - start_t):
    t, num_coins_hold, money_so_far, next_action = s
    current_price = prices[t]
    next_price = prices[t + 1]
    if next_action:
        if next_action == BUY: # buy
            num_coins_hold += 1
            money_so_far -= current_price
            next_action = None
        if next_action == SELL: # sell
            num_coins_hold -= 1
            money_so_far += current_price
            next_action = None
    else:
        # take action based on dynamics of price
        if next_price > current_price: # price will increase, then buy to hedge, sell in next time step
            # buy now to hedge
            num_coins_hold += 1
            money_so_far -= current_price
            # sell in next time step
            next_action = SELL
        elif next_price < current_price: # price will decrease, then sell to gain, buy in next time step
            # sell to gain
            num_coins_hold -= 1
            money_so_far += current_price           
            # buy back in next time step
            next_action = BUY
        else:
            # don't do anything, take action based on price dynamics at next time step
            next_action = None
    
    s = (t + 1, num_coins_hold, money_so_far, next_action)
    print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)
    

t = 1001 - num_coins: -1 - money: 8.03 - next_action: -1
t = 1002 - num_coins: 0 - money: 0.71 - next_action: None
t = 1003 - num_coins: -1 - money: 8.01 - next_action: -1
t = 1004 - num_coins: 0 - money: 0.91 - next_action: None
t = 1005 - num_coins: 1 - money: -6.67 - next_action: 1
t = 1006 - num_coins: 0 - money: 1.1 - next_action: None
t = 1007 - num_coins: 1 - money: -6.68 - next_action: 1
t = 1008 - num_coins: 0 - money: 1.12 - next_action: None
t = 1009 - num_coins: 1 - money: -6.74 - next_action: 1
t = 1010 - num_coins: 0 - money: 1.87 - next_action: None
t = 1011 - num_coins: -1 - money: 10.77 - next_action: -1
t = 1012 - num_coins: 0 - money: 2.15 - next_action: None
t = 1013 - num_coins: -1 - money: 10.71 - next_action: -1
t = 1014 - num_coins: 0 - money: 2.38 - next_action: None
t = 1015 - num_coins: 1 - money: -6.28 - next_action: 1
t = 1016 - num_coins: 0 - money: 2.44 - next_action: None
t = 1017 - num_coins: -1 - money: 11.05 - next_action: -1
t = 1018 - num_coins: 0 -

t = 1369 - num_coins: -1 - money: 26.55 - next_action: -1
t = 1370 - num_coins: 0 - money: 21.3 - next_action: None
t = 1371 - num_coins: 1 - money: 14.94 - next_action: 1
t = 1372 - num_coins: 0 - money: 22.47 - next_action: None
t = 1373 - num_coins: -1 - money: 26.95 - next_action: -1
t = 1374 - num_coins: 0 - money: 22.56 - next_action: None
t = 1375 - num_coins: -1 - money: 27.86 - next_action: -1
t = 1376 - num_coins: 0 - money: 23.18 - next_action: None
t = 1377 - num_coins: -1 - money: 27.83 - next_action: -1
t = 1378 - num_coins: 0 - money: 23.63 - next_action: None
t = 1379 - num_coins: 1 - money: 19.91 - next_action: 1
t = 1380 - num_coins: 0 - money: 23.69 - next_action: None
t = 1381 - num_coins: -1 - money: 27.69 - next_action: -1
t = 1382 - num_coins: 0 - money: 23.92 - next_action: None
t = 1383 - num_coins: -1 - money: 27.85 - next_action: -1
t = 1384 - num_coins: 0 - money: 24.07 - next_action: None
t = 1385 - num_coins: -1 - money: 27.9 - next_action: -1
t = 1386 - n

In [56]:
#t = 2000 - num_coins: 0 - money: 48.6 - next_action: None. Which is much higher than all other policies

In [81]:
# create the training dataset
# X = [price1, price2, price3, ..., price 10] (m x 10) where m = number of training examples
# Y = (m, ) array of 1: increase, 0: stays the same, -1: decrease

X = []
y = []

t = start_t
m = max_t - start_t
for i in range(m): #(max_t - start_t):
    past_ten_prices = prices[t-9: t+1]
    current_price = prices[t]
    next_price = prices[t + 1]
    label = 0
    if next_price > current_price:
        label = 1
    elif next_price < current_price:
        label = -1
    # print 'past_10: {} - next: {} - label: {}'.format(past_ten_prices, next_price, label)
    t += 1
    X.append(past_ten_prices)
    y.append(label)
    
print X
print y
    

[array([ 9.61,  9.2 ,  9.09,  9.27,  9.2 ,  8.97,  8.96,  7.98,  8.07,  8.03]), array([ 9.2 ,  9.09,  9.27,  9.2 ,  8.97,  8.96,  7.98,  8.07,  8.03,  7.32]), array([ 9.09,  9.27,  9.2 ,  8.97,  8.96,  7.98,  8.07,  8.03,  7.32,  7.3 ]), array([ 9.27,  9.2 ,  8.97,  8.96,  7.98,  8.07,  8.03,  7.32,  7.3 ,  7.1 ]), array([ 9.2 ,  8.97,  8.96,  7.98,  8.07,  8.03,  7.32,  7.3 ,  7.1 ,  7.58]), array([ 8.97,  8.96,  7.98,  8.07,  8.03,  7.32,  7.3 ,  7.1 ,  7.58,  7.77]), array([ 8.96,  7.98,  8.07,  8.03,  7.32,  7.3 ,  7.1 ,  7.58,  7.77,  7.78]), array([ 7.98,  8.07,  8.03,  7.32,  7.3 ,  7.1 ,  7.58,  7.77,  7.78,  7.8 ]), array([ 8.07,  8.03,  7.32,  7.3 ,  7.1 ,  7.58,  7.77,  7.78,  7.8 ,  7.86]), array([ 8.03,  7.32,  7.3 ,  7.1 ,  7.58,  7.77,  7.78,  7.8 ,  7.86,  8.61]), array([ 7.32,  7.3 ,  7.1 ,  7.58,  7.77,  7.78,  7.8 ,  7.86,  8.61,  8.9 ]), array([ 7.3 ,  7.1 ,  7.58,  7.77,  7.78,  7.8 ,  7.86,  8.61,  8.9 ,  8.62]), array([ 7.1 ,  7.58,  7.77,  7.78,  7.8 ,  7.86,  8

In [82]:
# split train test
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [83]:
# now build a neural net that given a price will give whether price will increase or decrease compared to current price?
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 500), random_state=1)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_train)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train, y_predict)
print 'Training accuracy: {}'.format(accuracy)

Training accuracy: 0.526536312849


In [84]:
# now build a neural net that given a price will give whether price will increase or decrease compared to current price?
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 500), random_state=1)
clf.fit(X_train, y_train)
y_test_predict = clf.predict(X_test)
print y_test_predict
print y_test

[-1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1, 1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 0, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 0, -1, 0, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 0, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -

In [85]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_test_predict)
print 'Test accuracy: {}'.format(accuracy)

Test accuracy: 0.4875


In [ ]:
# train on:
# t = start_t
# m = max_t - start_t
# Training accuracy: 0.526536312849
# solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 500), random_state=1
# Test accuracy: 0.4875

In [87]:
import pickle
# save the model to disk
filename = 'naive_neuralnet.sav'
pickle.dump(clf, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

In [88]:
print result

0.4875


In [94]:
# Use the trained neural net to trade

s = (start_t, 0, 0, None) # (t, num_coins_hold, money_so_far, next_action); 
# next_action = None: then take action based on dynamics of price, else take the specified next action
# action: -1: buy, 1: sell, 0: hold (probably not used for now)
for i in range(1000): #(max_t - start_t):
    t, num_coins_hold, money_so_far, next_action = s
    current_price = prices[t]
    ten_prices = prices[t-9: t+1]
#     next_price = prices[t + 1]
    if next_action:
        if next_action == BUY: # buy
            num_coins_hold += 1
            money_so_far -= current_price
            next_action = None
        if next_action == SELL: # sell
            num_coins_hold -= 1
            money_so_far += current_price
            next_action = None
    else:
        # take action based on dynamics of price
        if loaded_model.predict([ten_prices])[0] == 1: # price will increase, then buy to hedge, sell in next time step
            print loaded_model.predict([ten_prices])[0]
            # buy now to hedge
            num_coins_hold += 1
            money_so_far -= current_price
            # sell in next time step
            next_action = SELL
        elif loaded_model.predict([ten_prices])[0] == -1: # price will decrease, then sell to gain, buy in next time step
            print loaded_model.predict([ten_prices])[0]
            # sell to gain
            num_coins_hold -= 1
            money_so_far += current_price           
            # buy back in next time step
            next_action = BUY
        else:
            # don't do anything, take action based on price dynamics at next time step
            next_action = None
    
    s = (t + 1, num_coins_hold, money_so_far, next_action)
    #print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)
   
print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

In [ ]:
#t = 2000 - num_coins: 0 - money: 8.44 - next_action: None (range 1000)

In [ ]:
# also why the neural net keeps outputing negative values?

In [120]:
# TODO: change to run 100-timestep epsisodes
def generate_NN_1_step_policy_ideal(start_time, prices, policy_length=100):
    policy = []
    s = (start_time, 0, 0, None) # (t, num_coins_hold, money_so_far, next_action); 
    # next_action = None: then take action based on dynamics of price, else take the specified next action
    # action: -1: buy, 1: sell, 0: hold (probably not used for now)
    for i in range(policy_length): #(max_t - start_t):
        t, num_coins_hold, money_so_far, next_action = s
        current_price = prices[t]
        ten_prices = prices[t-9: t+1]
        next_price = prices[t + 1]
        if next_action:
            if next_action == BUY: # buy
                policy.append(BUY)
                num_coins_hold += 1
                money_so_far -= current_price
                next_action = None
            if next_action == SELL: # sell
                policy.append(SELL)
                num_coins_hold -= 1
                money_so_far += current_price
                next_action = None
        else:
            # take action based on dynamics of price
            if next_price > current_price: # price will increase, then buy to hedge, sell in next time step
                policy.append(BUY)
                #print loaded_model.predict([ten_prices])[0]
                # buy now to hedge
                num_coins_hold += 1
                money_so_far -= current_price
                # sell in next time step
                next_action = SELL
            elif next_price > current_price: # price will decrease, then sell to gain, buy in next time step
                policy.append(SELL)
                #print loaded_model.predict([ten_prices])[0]
                # sell to gain
                num_coins_hold -= 1
                money_so_far += current_price           
                # buy back in next time step
                next_action = BUY
            else:
                # don't do anything, take action based on price dynamics at next time step
                next_action = None
                policy.append(HOLD)
        s = (t + 1, num_coins_hold, money_so_far, next_action)
        #print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)
    return policy
    print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)

In [121]:
# TODO: change to run 100-timestep epsisodes
def generate_NN_1_step_policy(start_time, prices, policy_length=100):
    policy = []
    s = (start_time, 0, 0, None) # (t, num_coins_hold, money_so_far, next_action); 
    # next_action = None: then take action based on dynamics of price, else take the specified next action
    # action: -1: buy, 1: sell, 0: hold (probably not used for now)
    for i in range(policy_length): #(max_t - start_t):
        t, num_coins_hold, money_so_far, next_action = s
        current_price = prices[t]
        ten_prices = prices[t-9: t+1]
    #     next_price = prices[t + 1]
        if next_action:
            if next_action == BUY: # buy
                policy.append(BUY)
                num_coins_hold += 1
                money_so_far -= current_price
                next_action = None
            if next_action == SELL: # sell
                policy.append(SELL)
                num_coins_hold -= 1
                money_so_far += current_price
                next_action = None
        else:
            # take action based on dynamics of price
            # print 'ten_prices: {}'.format(ten_prices)
            if loaded_model.predict([ten_prices])[0] == 1: # price will increase, then buy to hedge, sell in next time step
                policy.append(BUY)
                #print loaded_model.predict([ten_prices])[0]
                # buy now to hedge
                num_coins_hold += 1
                money_so_far -= current_price
                # sell in next time step
                next_action = SELL
            elif loaded_model.predict([ten_prices])[0] == -1: # price will decrease, then sell to gain, buy in next time step
                policy.append(SELL)
                #print loaded_model.predict([ten_prices])[0]
                # sell to gain
                num_coins_hold -= 1
                money_so_far += current_price           
                # buy back in next time step
                next_action = BUY
            else:
                # don't do anything, take action based on price dynamics at next time step
                next_action = None
                policy.append(HOLD)
        s = (t + 1, num_coins_hold, money_so_far, next_action)
        #print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)
    return policy
    print 't = {} - num_coins: {} - money: {} - next_action: {}'.format(t + 1, num_coins_hold, money_so_far, next_action)

In [122]:
import evaluate

In [157]:
import random
prices = hp.get_price_history().squeeze()

reward_array = []
for i in range(hp.get_evaluation_step()):
    random_time = random.randint(1000, 2000)
    policy = generate_NN_1_step_policy(random_time, prices, 100)
    #print 'Policy: {}'.format(policy)
    reward = evaluate.evaluate_reward(policy, prices, random_time)
    reward_array.append(reward)

evaluate.print_statistics_95_confidence(reward_array)

('average:', 0.43433333333333379, 'standard_error', 0.84516573894556613)


In [167]:
prices = hp.get_price_history().squeeze()

def upper_bound_of_maximum_gain(time_step, prices, horizon=100):
    #all_prices_in_100_time_step = np.array(prices[time_step: time_step + horizon])
    all_prices_in_100_time_step = np.array(prices[time_step: time_step + horizon])
    closing_price = all_prices_in_100_time_step[-1]
    diff = all_prices_in_100_time_step - closing_price
    return np.sum(np.absolute(diff))
    
def upper_bound_of_maximum_gain_50(time_step, prices, horizon=100):
    #all_prices_in_100_time_step = np.array(prices[time_step: time_step + horizon])
    all_prices_in_100_time_step = np.array(prices[time_step: time_step + horizon])
    closing_price = all_prices_in_100_time_step[-1]
    diff = all_prices_in_100_time_step - closing_price
    diff_50 = diff[:50]
    return np.sum(np.absolute(diff_50))    
    

In [171]:
import random
prices = hp.get_price_history().squeeze()

reward_array = []
for i in range(hp.get_evaluation_step()):
    random_time = random.randint(1000, 2000)
    policy = generate_NN_1_step_policy(random_time, prices, 100)
    #policy = generate_NN_1_step_policy_ideal(random_time, prices, 100)
    #print 'Policy: {}'.format(policy)
    reward = evaluate.evaluate_reward(policy, prices, random_time)
    upper_bound_of_max_gain = upper_bound_of_maximum_gain(random_time, prices)
    upper_bound_of_max_gain_50 = upper_bound_of_maximum_gain_50(random_time, prices)
    print 'Reward: {} - Upper bound: {} - Upper bound first 50: {}'.format(reward, upper_bound_of_max_gain, upper_bound_of_max_gain_50)
    reward_array.append(reward)

evaluate.print_statistics_95_confidence(reward_array)

Reward: -0.94 - Upper bound: 50.6 - Upper bound first 50: 28.34
Reward: 3.16 - Upper bound: 548.92 - Upper bound first 50: 309.58
Reward: 1.17 - Upper bound: 33.27 - Upper bound first 50: 21.3
Reward: 0.63 - Upper bound: 96.33 - Upper bound first 50: 57.13
Reward: -1.65 - Upper bound: 106.87 - Upper bound first 50: 60.88
Reward: 0.83 - Upper bound: 52.97 - Upper bound first 50: 46.59
Reward: 0.58 - Upper bound: 28.36 - Upper bound first 50: 13.18
Reward: 1.1 - Upper bound: 34.6 - Upper bound first 50: 30.14
Reward: -0.27 - Upper bound: 31.73 - Upper bound first 50: 27.0
Reward: 1.04 - Upper bound: 19.68 - Upper bound first 50: 13.62
Reward: -1.14 - Upper bound: 170.64 - Upper bound first 50: 116.09
Reward: -1.24 - Upper bound: 45.66 - Upper bound first 50: 28.85
Reward: 2.42 - Upper bound: 109.18 - Upper bound first 50: 92.43
Reward: -0.61 - Upper bound: 37.63 - Upper bound first 50: 29.06
Reward: 0.28 - Upper bound: 104.88 - Upper bound first 50: 87.44
Reward: 0.87 - Upper bound: 18.3